In [0]:
%sql
CREATE CONNECTION IF NOT EXISTS practice1_earthquake_conn_updated
type HTTP
OPTIONS (
  host = 'https://earthquake.usgs.gov',
  port = 443,
  base_path = '/earthquakes/feed/v1.0/',
  bearer_token = 'na'
) 

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

conn = w.connections.get("practice1_earthquake_conn_updated")

base_url = f"{conn.options['host']}{conn.options['base_path']}"

In [0]:
dbutils.widgets.text('catalog_name','end_to_end_practice_1','end_to_end_practice_1')
catalog_name = dbutils.widgets.get('catalog_name')

In [0]:
spark.sql(
    f"use catalog {catalog_name}"
)
spark.sql(
    "use schema bronze"
)
spark.sql(
    "create volume if not exists earthquake_data"
)

In [0]:
import requests
import json
import datetime

url = f"{base_url}/summary/all_day.geojson"
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Request failed with status code {url}")
data = response.json()
current_date=datetime.datetime.now().strftime("%Y-%m-%d")
dbutils.fs.put(
    f"/Volumes/end_to_end_practice_1/bronze/earthquake_data/earthquake_data{current_date}.json",
    json.dumps(data),
    overwrite=True
)

In [0]:
import datetime
current_date=datetime.datetime.now().strftime("%Y-%m-%d")

import json

file_path = f"/Volumes/end_to_end_practice_1/bronze/earthquake_data/earthquake_data{current_date}.json"

with open(file_path, "r") as f:
    data_read = json.load(f)

print(data_read)


In [0]:
# Pretty print
print(json.dumps(data_read, indent=2))